## Import the libraries

In [19]:
import tensorflow.keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import sys
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
##load the vgg model
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## freeze the entire model, unfreeze the final block Block5

In [2]:
#freeze the entire model, unfreeze the final block

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.summary()

## use the image data generator to generate artificial samples 
## compile the model with stochastic gradient descent

In [3]:
## use image data generator
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   width_shift_range=0.1, height_shift_range=0.1,
                                    horizontal_flip=True,rotation_range=20
                                    ,brightness_range=[0.2,1.2],
                                    shear_range=0.01)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = train_datagen.flow_from_directory('data/train/',
                                             class_mode="categorical", batch_size=32,
                                              target_size=(150, 150))
test_it = test_datagen.flow_from_directory('data/test/',
class_mode="categorical", batch_size=32, target_size=(150, 150))
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print(len(train_it))
print(len(test_it))

Found 2253 images belonging to 6 classes.
Found 274 images belonging to 6 classes.
71
9


## train the model while monitoring validation accuracy to be maximum

In [4]:
checkpoint = ModelCheckpoint('model1.h5', monitor='val_acc', save_best_only=True, mode='max')
history = model.fit_generator(train_it, steps_per_epoch=140,
                              validation_data=test_it, validation_steps=20, callbacks=[checkpoint], epochs=25)
model.save('model1.h5')

Epoch 1/25
140/140 [==============================] - 223s 2s/step - loss: 1.2439 - acc: 0.5050 - val_loss: 1.0682 - val_acc: 0.5866
Epoch 2/25
140/140 [==============================] - 224s 2s/step - loss: 0.8442 - acc: 0.6731 - val_loss: 1.0426 - val_acc: 0.6422
Epoch 3/25
140/140 [==============================] - 223s 2s/step - loss: 0.6549 - acc: 0.7577 - val_loss: 1.2044 - val_acc: 0.6046
Epoch 4/25
140/140 [==============================] - 222s 2s/step - loss: 0.6209 - acc: 0.7791 - val_loss: 1.1363 - val_acc: 0.6062
Epoch 5/25
140/140 [==============================] - 222s 2s/step - loss: 0.4880 - acc: 0.8138 - val_loss: 1.0698 - val_acc: 0.6830
Epoch 6/25
140/140 [==============================] - 222s 2s/step - loss: 0.4428 - acc: 0.8451 - val_loss: 1.1969 - val_acc: 0.6405
Epoch 7/25
140/140 [==============================] - 222s 2s/step - loss: 0.3578 - acc: 0.8721 - val_loss: 1.2343 - val_acc: 0.6716
Epoch 8/25
140/140 [==============================] - 223s 2s/step - 

# Architecture of LeNet 5 is as follows

## INPUT LAYER

## C1 layer-convolutional layer

## S2 layer-pooling layer (downsampling layer)

## C3 layer-convolutional layer

## S4 layer-pooling layer (downsampling layer)

## C5 layer-convolution layer

## F6 layer-fully connected layer

## Output layer-fully connected layer


In [12]:
def lenet():
    # Building the Model Architecture
    model = Sequential()
    ##   150 ∗ 150 = 22500.
    # Parameters between input layer and C1 layer: 6 ∗ (5 ∗ 5 + 1)
    model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(150, 150, 3)))

    # The size of the filter used in this layer is 2 * 2, and the step length and width are both 2,
    ## so the output matrix size of this layer is 75 * 75 * 6.
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # This layer has 5 * 5 * 6 * 16 + 16 = 2416 parameters
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))

    # The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, 
    ##so the output matrix size of this layer is 5 * 5 * 16.
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))

    # The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(84, activation='relu'))

    # The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(6, activation='softmax'))
    
    return model

## train with stochastic gradient descent 

In [13]:
model2 = lenet()
opt = SGD(lr=0.001, momentum=0.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 146, 146, 6)       456       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 6)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 69, 69, 16)        2416      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 34, 34, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18496)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 120)               2219640   
_________________________________________________________________
dense_12 (Dense)             (None, 84)               

## train the model

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model2.h5', monitor='val_acc', save_best_only=True, mode='max')
history = model2.fit_generator(train_it, steps_per_epoch=140,
                              validation_data=test_it, validation_steps=9, callbacks=[checkpoint], epochs=25)
model2.save('model2.h5')

Epoch 1/25
140/140 [==============================] - 58s 413ms/step - loss: 1.7199 - acc: 0.2456 - val_loss: 1.6813 - val_acc: 0.2482
Epoch 2/25
140/140 [==============================] - 44s 315ms/step - loss: 1.6141 - acc: 0.3476 - val_loss: 1.7157 - val_acc: 0.2701
Epoch 3/25
140/140 [==============================] - 46s 330ms/step - loss: 1.5406 - acc: 0.3778 - val_loss: 1.5559 - val_acc: 0.4161
Epoch 4/25
140/140 [==============================] - 44s 315ms/step - loss: 1.4595 - acc: 0.4259 - val_loss: 1.5490 - val_acc: 0.3504
Epoch 5/25
140/140 [==============================] - 53s 381ms/step - loss: 1.4070 - acc: 0.4526 - val_loss: 1.4509 - val_acc: 0.4161
Epoch 6/25
140/140 [==============================] - 48s 345ms/step - loss: 1.3458 - acc: 0.4773 - val_loss: 1.5412 - val_acc: 0.3942
Epoch 7/25
140/140 [==============================] - 44s 317ms/step - loss: 1.2922 - acc: 0.5106 - val_loss: 1.4327 - val_acc: 0.4234
Epoch 8/25
140/140 [==============================] - 4

## strategy for resnet is different with different train and test splits according to its orignal respective files

In [36]:
import re
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def list_dataset():
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

            
# Add class name prefix to each path based on class name include in filename
def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
    return df


def class_id_to_label(id):
    label_map = {1: 'glass', 2: 'paper', 3: 'cardboard', 4: 'plastic', 5: 'metal', 6: 'trash'}
    return label_map[id]
    

IMAGES_DIR = 'Garbageclassification/'
    
train_file = 'one-indexed-files-notrash_train.txt'
val_file   = 'one-indexed-files-notrash_val.txt'
test_file  = 'one-indexed-files-notrash_test.txt'

df_train = pd.read_csv(train_file, sep=' ', header=None, names=['rel_path', 'label'])
df_valid = pd.read_csv(val_file,   sep=' ', header=None, names=['rel_path', 'label'])
df_test  = pd.read_csv(val_file,   sep=' ', header=None, names=['rel_path', 'label'])

df_train = add_class_name_prefix(df_train, 'rel_path')
df_valid = add_class_name_prefix(df_valid, 'rel_path')
df_test  = add_class_name_prefix(df_test,  'rel_path')

df_train['label'] = df_train['label'].apply(class_id_to_label)
df_valid['label'] = df_valid['label'].apply(class_id_to_label)
df_test['label']  = df_test['label'].apply(class_id_to_label)

print(f'Found {len(df_train)} training, {len(df_valid)} validation and {len(df_test)} samples.')

Found 1768 training, 328 validation and 328 samples.


In [37]:
df_train.head()

,rel_path,label
0,cardboard/cardboard202.jpg,cardboard
1,paper/paper472.jpg,paper
2,paper/paper522.jpg,paper
3,glass/glass189.jpg,glass
4,glass/glass325.jpg,glass


## Initialize the image data generator with some modes 

In [38]:
datagen = ImageDataGenerator()

datagen_train = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=IMAGES_DIR,
    x_col='rel_path',
    y_col='label',
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=7,
)

datagen_valid = datagen.flow_from_dataframe(
    dataframe=df_valid,
    directory=IMAGES_DIR,
    x_col='rel_path',
    y_col='label',
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=7,
)

Found 1768 validated image filenames belonging to 6 classes.
Found 328 validated image filenames belonging to 6 classes.


## Build the ResNet model

In [39]:
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False
        
    return model


net = build_model(num_classes=6)

net.compile(optimizer='Adam',
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.categorical_accuracy])

net.summary()

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

history = net.fit_generator(
    generator=datagen_train,
    validation_data=datagen_valid,
    epochs=30,
    validation_freq=1,
    callbacks=[early_stop]
)

E:\condaaa\envs\tf\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

Epoch 1/30
56/56 [==============================] - 630s 11s/step - loss: 1.0922 - categorical_accuracy: 0.6544 - val_loss: 0.9520 - val_categorical_accuracy: 0.6555
Epoch 2/30
56/56 [==============================] - 615s 11s/step - loss: 0.3860 - categorical_accuracy: 0.8563 - val_loss: 1.0005 - val_categorical_accuracy: 0.6555
Epoch 3/30
56/56 [==============================] - 663s 12s/step - loss: 0.2960 - categorical_accuracy: 0.8908 - val_loss: 1.0342 - val_categorical_accuracy: 0.6585
Epoch 4/30
56/56 [==============================] - 692s 12s/step - loss: 0.2420 - categorical_accuracy: 0.9135 - val_loss: 0.9842 - val_categorical_accuracy: 0.6707
Epoch 5/30
56/56 [==============================] - 719s 13s/step - loss: 0.1457 - categorical_accuracy: 0.9497 - val_loss: 0.9481 - val_categorical_accuracy: 0.7317
Epoch 6/30
56/56 [==============================] - 692s 12s/step - loss: 0.0913 - categorical_accuracy: 0.9729 - val_loss: 1.3928 - val_categorical_accuracy: 0.6433
Epoc

In [40]:
net.save('model3.h5')

## For model 1 and 2 only

In [ ]:
# make a prediction for a new image.
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(150, 150))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 150, 150, 3)
	# center pixel data
	img = img.astype('float32')
	img= img/255
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('Garbageclassification/cardboard/cardboard12.jpg')
	# load model
	model = load_model('model1.h5')
	# predict the class
	result = model.predict(img)
	result = result.argmax()
	print(result)
	if result == 0:
		print('cardboard')
	if result == 1:
		print('glass')
	if result == 2:
		print('metal')
	if result == 3:
		print('paper')
	if result == 4:
		print('plastic')
	if result == 5:
		print('trash')

# entry point, run the example
run_example()

In [ ]:
# make a prediction for a new image.
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(150, 150))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 150, 150, 3)
	# center pixel data
	img = img.astype('float32')
	img= img/255
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('Garbageclassification/cardboard/cardboard12.jpg')
	# load model
	model = load_model('model2.h5')
	# predict the class
	result = model.predict(img)
	result = result.argmax()
	print(result)
	if result == 0:
		print('cardboard')
	if result == 1:
		print('glass')
	if result == 2:
		print('metal')
	if result == 3:
		print('paper')
	if result == 4:
		print('plastic')
	if result == 5:
		print('trash')
# entry point, run the example
run_example()